# Imports

In [17]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
import os.path
import csv
from pathlib import Path
from pandas_datareader import data
# Don't use all these modules, but oh well.

# function: pickle_ticker_data_for_date_range()

This function will give you the HOLC for the date range passed, if the data for this specif time period was already pulled (and saved into a pickle file) a Pandas Data Frame will be returned with that data. If the data doesn't exist (like the first time you run this) the data will get pulled by the amazing `pandas_datareader`

In [18]:
def pickle_ticker_data_for_date_range(ticker,start,end):
    """gimme a ticker, a start and end date, and I will return a Pandas DataFrame with that data. """
    pickle_file = f"../../data/rawdata/stock-data-by-range/{start}_{end}/{ticker}.pickle"
    df = None
    if os.path.exists(pickle_file):
        df = pickle.load( open( pickle_file, "rb" ) )
    else:
        df = data.DataReader(ticker,'yahoo',start,end)
        #.filter(['Date','Adj Close'])
        #df.rename(columns={'Adj Close': ticker},inplace=True)
        pickle.dump( df, open( pickle_file, "wb" ) )
    return df

# Example usage with a single ticker: AAPL

In [16]:
# Set the ticker value
# and start and end
ticker = "AAPL"
start = '2006-01-01'
end = '2018-12-31'

# Call the function
aapl_df = pickle_ticker_data_for_date_range(ticker,start,end)

# Print the firest few rows (notice the date) head() will confirm the end date
aapl_df.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2006-01-03,10.678572,10.321428,10.340000,10.678572,201808600.0,9.319328
2006-01-04,10.854285,10.642858,10.732857,10.710000,154900900.0,9.346760
2006-01-05,10.700000,10.535714,10.690000,10.625714,112355600.0,9.273201
2006-01-06,10.957143,10.650000,10.750000,10.900000,176114400.0,9.512576
2006-01-09,11.028571,10.820000,10.961429,10.864285,168760200.0,9.481407


# Example usage with a List of tickers

In [27]:
tickers = ['SPY','AGG']
start = '2006-01-01'
end = '2018-12-31'

combined_df = None

for ticker in tickers:
    prefix = ticker + '_'
    close = prefix + "Close"
    df = pickle_ticker_data_for_date_range(ticker,start,end)
    df = df.add_prefix(prefix)
    #print(df[close].head())
    # Let's just look at close for now...
    combined_df = pd.concat([combined_df, df.filter([close])], axis="columns", join="inner")
combined_df

,SPY_Close,AGG_Close
Date,,
2006-01-03,126.699997,100.639999
2006-01-04,127.300003,100.879997
2006-01-05,127.379997,100.879997
2006-01-06,128.440002,100.650002
2006-01-09,128.770004,100.760002
...,...,...
2018-12-24,234.339996,105.860001
2018-12-26,246.179993,105.660004
2018-12-27,248.070007,105.830002
